# Larger systems

In this chapter we will consider some aspects which are important for considering larger systems. These include:

- Ways to lower computational costs (cost-saving measures)

- The inclusion of an environment

- Coupling to molecular dynamics simulations

- ...

Most of the examples provided here are have been calculated on a larger cluster, so directly running the scripts on a laptop is not advised. They are here included as static core, with results taken from separate calculations.


## Valence-to-core XES

Here considering transition metal complex with low degree of static correlation, such that TDDFT and ADC can be used.

Find experimental measurements and consider using

1. Overlap

2. TDDFT (?)

3. ADC (?)


## X-ray absorption spectrum of solvated system

### Uracil with CVS-ADC(2)-x

Consider uracil (or some similar system) with CVS-ADC(2)-x. Consider subjects such as 

1. Tailor CVS space

2. Gas phase spectra

3. Spectra with PE

### Phthalocyanine with TDDFT

Consider phthalocyanine (or some similar system) with TDDFT. Focus on just making it work, so not necessarily including an environment.


## X-ray emission spectrum of solvated system

### Uracil with ADC(2)

Consider uracil (or some similar system) with ADC(2) / ADC(3/2).

1. Gas phase

2. Environment with PE

Also could discuss the impact of double-excitations, which may necessitate ADC(3/2)

### Phthalocyanine with DFT

Consider phthalocyanine (or some similar system) with TDDFT and DFT overlap (compare and contrast). Focus on just making it work, so not necessarily including an environment.


## Coupling to molecular dynamics

Maybe include this, although it may be tricky. Examples could include:

1. Dynamics + XAS

2. Dynamics + XES

3. Inclusion of environment

4. Core-hole dynamics with some model


## Transient X-ray spectroscopy

```{note}
A long-term goal, which might be given its own section. Examples include using a two-step approach or coupling scheme for XAS, ...
```

# Transient

## Low-spin open-shell reference (LSOR)

Here an electron from an occupied alpha spin-orbital is promoted to an unoccupied alpha spin-orbital. Note that spin-contaminated by construction, so should perhaps focus on HSOR... Check S2 and so on, consider what would be needed to be spin-pure

- Include practical example with LSOR and HSOR

In [ ]:
# Single valence excitation LSOR

# Perform unrestricted SCF calculation
scf_gs = scf.UHF(mol)
scf_gs.kernel()

# Get MO coefficients and occupations
mo0  = copy.deepcopy(scf_gs.mo_coeff)
occ0 = copy.deepcopy(scf_gs.mo_occ)

# Do single valence excitation by moving 1 alpha electron from HOMO to LUMO
occ0[0][lumo-1] = 0.0
occ0[0][lumo] = 1.0

# Perform unrestricted SCF calculation on valence excited state
scf_es = scf.UHF(mol)
imom(scf_es, mo0, occ0)
scf_es.kernel()

# Calculate X-ray absorption spectrum
adc_xas = adcc.cvs_adc2(scf_es, n_states=4, core_orbitals=1)
print(adc_xas.describe())

# Plot tr-XAS
plt.figure(figsize=(6,5))

# Convolute using functionalities available in gator and adcc
plt.subplot(211); plt.title('tr-XAS (from singly valence excited state)')
adc_xas.plot_spectrum()
plt.tight_layout(); plt.show()

## XAS in other ways

### From electron affinity

Electron affinity (EA) is the energy change when a system gains an additional electron. One can model XAS as the result of first a core ionization, and then an addition of one valence electron. Thus, core excitation energies can in principle be determined as the sum of the core IP and the electron affinity. This is shown below.

In [ ]:
# Create pyscf mol object
mol       = gto.Mole()
mol.atom  = water_xyz
mol.basis = '6-31G'
mol.build()

# Perform unrestricted SCF calculation
scf_gs = scf.UHF(mol)
scf_gs.kernel()
adc_gs  = adc.UADC(scf_gs).run()

# Copy molecular orbitals
mo0        = copy.deepcopy(scf_gs.mo_coeff)
occ0       = copy.deepcopy(scf_gs.mo_occ)

# Create 1s core-hole by setting alpha_0 population to zero
occ0[0][0] = 0.0

# Perform unrestricted SCF calculation with MOM constraint
scf_ion = scf.UHF(mol)
scf.addons.mom_occ(scf_ion, mo0, occ0)
scf_ion.kernel()

# Perform ADC calculation of first four states
adc_ion = adc.UADC(scf_ion)
adc_ion.method = 'adc(2)-x'
adc_ion.run()
adc_ip = (adc_ion.scf_energy + adc_ion.e_corr - (adc_gs.scf_energy + adc_gs.e_corr))

#EA ADC ion
e_ea,v_ea,p_ea,x_ea,adc_ea = adc_ion.ea_adc(nroots = 4)

# XAS energies
xas_e = []
for i in range(len(e_ea)):
    xas_e.append(adc_ip + e_ea[i])

print(f'Excitation Energy/eV     spectral factor')
print(f'-------------------------------------------')
for i in range(len(xas_e)):
    print(f'   {xas_e[i]*au2ev : .2f}                   {p_ea[i] : .6f}')
print(f'-------------------------------------------')

# Plot with Lorentzian broadening
plt.figure(figsize=(6,5))
plt.subplot(211); plt.title('ADC(2)-X XAS from EA')
xi,yi = lorentzian(xas_e, p_ea,min(xas_e)-0.05, max(xas_e)+0.05,0.001,0.3/au2ev); plt.plot(xi*au2ev,yi)

plt.tight_layout(); plt.show()

We observe two sets of closelying peaks. Each set most likely is the result of the computation of both singlet and triplet state generation by adding an electron. The general position of the peaks, i. e. at 537.6 eV and 539.8 eV are shifted by approximately 1 eV compared to the CVS eigenstate values predicted in the echem notebook (not included here). The discrepancy could be a result of the different ways of describing the process. Whereas the "normal" CVS approach focus on the coupling between the core and valence states, this approach focus mainly on the valence states. Note that the plotted intensities do not carry information on oscillator strengths.

### From deexcitation from core excited states

One can also simply consider the XAS process in reverse, i.e. preparing a core excited state and determining the deexcitation energy and oscillator strength. 

In [ ]:
# Create pyscf mol object
mol       = gto.Mole()
mol.atom  = water_xyz
mol.basis = '6-31G'
mol.build()

xas_E = []
xas_f = []

# Perform unrestricted SCF calculation
scf_res = scf.UHF(mol)
scf_res.kernel()

# Copy molecular orbital coefficients
mo0        = copy.deepcopy(scf_res.mo_coeff)

# Create 1s core-hole by setting alpha_0 population to zero

for i in range(2):
# Copy molecular occupations and core excite from 1s alpha to a valence state
    occ0       = copy.deepcopy(scf_res.mo_occ)
    occ0[0][0] = 0.0
    occ0[0][5+i] = 1.0

# Perform unrestricted SCF calculation with MOM constraint
    scf_ion = scf.UHF(mol)
    scf.addons.mom_occ(scf_ion, mo0, occ0)
    scf_ion.kernel()

# Perform ADC calculation of first four states
    adc_xas = adcc.adc2x(scf_ion, n_states = 1)
    xas_E.append(-au2ev*adc_xas.excitation_energy) 
    xas_f.append(adc_xas.oscillator_strength)


print(f'\nExcitation Energy/eV     Osc. strength')
print(f'-------------------------------------------')
for i in range(len(xas_E)):
    print(f'   {xas_E[i][0] : .2f}                   {xas_f[i][0] : .6f}')
print(f'-------------------------------------------')

# Plot
plt.figure(figsize=(6,5))
plt.subplot(212); plt.title('ADC(2)-X XAS from MOM reference')
x,y = xas_E, xas_f
xi,yi = lorentzian(x,y,536,545,0.01,0.5); plt.plot(xi,yi)
plt.tight_layout(); plt.show()

We observe again a shift from the CVS approach (2 eV for the low energy peak). Observe that the peak splitting is in good agreement with both the CVS and the EA based results (approximately 2 eV). Also the the relative intensities are reproduced compared to the CVS result (the EA based result gives no indication of intensities). The small differences occur because we consider the process reversed and thus our reference is an optimized core excited state rather than an unperturbed GS.

## XES in other ways

The XES process can like the XAS process be considered in reverse and thus calculated from core excitation of a valence ionized state, rather than from valence deexcitations of a core ionized state.

In [ ]:
# Create pyscf mol object
mol       = gto.Mole()
mol.atom  = water_xyz
mol.basis = '6-31G'
mol.build()

xes_E = []
xes_f = []

# Perform unrestricted SCF calculation
scf_res = scf.UHF(mol)
scf_res.kernel()

# Copy molecular orbital coefficients
mo0        = copy.deepcopy(scf_res.mo_coeff)

# Create valence-hole by setting alpha population to zero
for i in range(4):
    occ0       = copy.deepcopy(scf_res.mo_occ)
    occ0[0][4-i] = 0.0

# Perform unrestricted SCF calculation with MOM constraint
    scf_ion = scf.UHF(mol)
    scf.addons.mom_occ(scf_ion, mo0, occ0)
    scf_ion.kernel()

# Perform ADC calculation of first four states
    adc_xes = adcc.cvs_adc2(scf_ion, n_states = 1, core_orbitals=1)
    xes_E.append(au2ev*adc_xes.excitation_energy) 
    xes_f.append(adc_xes.oscillator_strength)


print(f'\nExcitation Energy/eV     Osc. strength')
print(f'-------------------------------------------')
for i in range(len(xes_E)):
    print(f'   {xes_E[i][0] : .2f}                   {xes_f[i][0] : .6f}')
print(f'-------------------------------------------')

# Plot
plt.figure(figsize=(6,5))
plt.subplot(212); plt.title('ADC(2) XES from MOM valence reference')
x,y = xes_E, xes_f
xi,yi = lorentzian(x,y,500,535,0.01,0.5); plt.plot(xi,yi)
plt.tight_layout(); plt.show()